# Project: Wrangle and Visualize Data
## Twitter _WeRateDogs_ Case

In [ ]:
# Setting up my environment by important core packages
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import seaborn as sns
import html5lib
import requests
import tweepy
import json
import glob
import csv
import os
import matplotlib.pyplot as plt
%matplotlib inline

# Gathering Data

In [ ]:
# Loading the twitter archived data into a pandas  DataFrame
df_1 = pd.read_csv('twitter-archive-enhanced.csv')

In [ ]:
# Let's download the HTML programmaticaly by sending a GET request to the url
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url, allow_redirects = True)

# Saving TSV to file
with open("image-predictions.tsv", mode = 'wb') as file:
    file.write(response.content)
    
soup = BeautifulSoup(response.content, 'lxml')

In [ ]:
df_2 = pd.read_csv('image-predictions.tsv', sep = '\t')

In [ ]:
# Loading the json file into a pandas DataFrame
df_3 = pd.read_json('tweet.json', lines = True)

# Assessing Data
### Visual Assessment

In [ ]:
# Display twitter-archive-enhanced Table
df_1

In [ ]:
# Display image-predictions table
df_2

In [ ]:
# Display tweet table
df_3

### Programmatic Assessment

In [ ]:
df_1.info()

In [ ]:
df_2.info()

In [ ]:
df_3.info()

In [ ]:
df_1.describe()

In [ ]:
df_2.describe()

In [ ]:
df_3.describe()

In [ ]:
df_1.sample(4)

In [ ]:
df_2.sample(10)

In [ ]:
df_3.sample(5)

In [ ]:
df_1.duplicated().sum()

In [ ]:
df_2.duplicated().sum()

In [ ]:
df_1.name.value_counts()

In [ ]:
df_2.jpg_url.value_counts()

In [ ]:
df_3.full_text.value_counts()

In [ ]:
df_3.source.value_counts()

In [ ]:
all_columns = pd.Series(list(df_1) + list(df_2) + list(df_3))
all_columns[all_columns.duplicated()]

### Quality
#### "tweeter-archive-enhanced" table
- Poorly formatted timestamp columns (timestamp, retweeted_status_timestamp)
- Column names aren't clear enough
- Confusing names (a, an)
- Erroneous datatypes (tweet_id, retweeted_status_id, retweeted_status_user_id, in_reply_to_status_id, in_reply_to_user_id)
- Presence of non null values in retweet_related columns (retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp)
- Missing of so many information in columns (in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp)
- Retweeted data aren't revelant to the analysis (retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp)
- Extraneous columns (source, text, expanded_urls)

#### "image-predictions" table
- Erroneous datatype (tweet_id)
- Column names aren't clear enough (p1, p2, p3, p1_conf, p2_dog)
- Inconsistency in column values capitalization (p1, p2, p3)
- Inconsistency in word separation in column values (some with hyphen (-) others with underscore(_))

#### "tweet" table
- Poorly formatted dates (created_at column name)
- Erroneous datatypes (id, id_str, in_reply_to_status_id, in_reply_to_status_id_str, in_reply_to_user_id, in_reply_to_user_id_str)
- Entities column contains curly braces and square brackets due to the json format
- Missing of huge amount of informations in columns (in_reply_to_status_id, in_reply_to_status_id_str, in_reply_to_user_id, in_reply_to_user_id_str, in_reply_to_screen_name, geo, coordinates, place, contributors, retweeted_status, quoted_status_id, quoted_status_id_str, quoted_status)
- Missing information in some columns (extented_entities, possibly_sensitive, possibly_sensitive_appealable)
- Erroneous datatypes (in_reply_to_status_id, in_reply_to_status_id_str, in_reply_to_user_id, in_reply_to_user_id_str)
- Column values equal to zero (possibly_sensitive, possibly_sensitive_appealable)
- Retweeted data aren't relevant to the analysis (
- tweets beyond August 1st, 2017 aren't relevant since their image predictions are not available

### Tidiness
#### "tweeter-archive-enhanced" table
- There is one variable in four columns, each dog's name variable does not form a column (doggo, fluffer, pupper, puppo)

#### "image-predictions" table
- Image predictions are split up into several lines
- Prediction, confidence and breed of dogs (p, p_conf, p_dog) do not respectively form a single column

#### "tweet" table


# Cleaning Data

In [ ]:
# Create a copy of the twitter-archive-enhanced dataframe
df_clean_1 = df_1.copy()

### Define
- Convert timestamp and retweeted_status_timestamp into datetime
- Rename name column into dog_name
- Remove confusing dogs' name
- Convert tweet_id, retweeted_status_id, retweeted_status_user_id, in_reply_to_status_id, in_reply_to_user_id into string datatype
- Remove rows that have values belonging to columns retweeted_status_id, retweeted_status_user_id, and retweeted_status_timestamp  to get rid of retweets
- Ditch columns with most of the values missing
- Drop extraneous columns that aren't relevant to our questions
- Merge the four columns (doggo, fluffer, pupper, puppo) into one variable called dog_name
- Remove extraneous columns (source, text, expanded_urls)

### Code

In [ ]:
# Convert timestamp and retweeted_status_timestamp into datetime
df_clean_1.timestamp = pd.to_datetime(df_clean_1.timestamp)

df_clean_1.retweeted_status_timestamp = pd.to_datetime(df_clean_1.retweeted_status_timestamp)

In [ ]:
# Rename name column into dog_name
df_clean_1.rename(columns = {'name': 'dog_name'}, inplace = True)

In [ ]:
# Remove confusing dogs' name
df_clean_1 = df_clean_1[df_clean_1.dog_name != 'a']
df_clean_1 = df_clean_1[df_clean_1.dog_name != 'an']

In [ ]:
# Convert tweet_id, retweeted_status_id, retweeted_status_user_id, in_reply_to_status_id, in_reply_to_user_id into string datatype
id_columns = ['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id']
for i in id_columns:
    df_clean_1[i] = df_clean_1[i].astype(str)

In [ ]:
#  Remove rows that have values belonging to columns retweeted_status_id, retweeted_status_user_id, and retweeted_status_timestamp  to get rid of retweets
df_clean_retweet = df_clean_1[df_clean_1.retweeted_status_id.isnull()]
df_clean_retweet = df_clean_1[df_clean_1.retweeted_status_user_id.isnull()]
df_clean_retweet = df_clean_1[df_clean_1.retweeted_status_timestamp.isnull()]

In [ ]:
# Ditch columns with most of the values missing and drop extraneous ones
drop_columns = ['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id','retweeted_status_timestamp', 'source', 'text', 'expanded_urls']
df_clean_retweet.drop(drop_columns, axis = 1, inplace = True)

In [ ]:
# Replace None with ''
dog_columns = ['doggo', 'floofer', 'pupper', 'puppo', 'dog_name']
for p in dog_columns:
    df_clean_retweet[p] = df_clean_retweet[p].str.replace('None', '', regex = True)

In [ ]:
# Merge the four columns (doggo, fluffer, pupper, puppo) into one variable called dog_name
df_clean_retweet['dog_stage'] = df_clean_retweet['doggo'] + df_clean_retweet['floofer'] + df_clean_retweet['pupper'] + df_clean_retweet['puppo']

In [ ]:
# Drop extraneous columns
drop_columns = ['doggo', 'floofer', 'pupper', 'puppo']
df_clean_retweet.drop(drop_columns, axis = 1, inplace = True)

# Fill empty columns with na
empty_columns = ['dog_name', 'dog_stage']
for e in empty_columns:
    df_clean_retweet[e] = df_clean_retweet[e].replace('', np.nan)

### Test

In [ ]:
df_clean_retweet.info()

In [ ]:
df_clean_retweet.duplicated().sum()

In [ ]:
df_clean_retweet.columns

In [ ]:
df_clean_retweet.dog_name.value_counts().sort_values()

In [ ]:
df_clean_retweet.head()

In [ ]:
df_clean_retweet.sample(5)

#### Create a copy of the image-predictions dataframe

In [ ]:
df_clean_2 = df_2.copy()

### Define
- Convert tweet_id into string datatype
- Rename columns p1, p2, p3
- Make capitalization consistent columns p1, p2, p3 values
- Replace hyphen with underscore sign in column values
- Merge prediction, confidence and breed variables into three columns

### Code

In [ ]:
# Convert tweet_id into string datatype
df_clean_2.fillna(0, inplace = True)
df_clean_2['tweet_id'] = df_clean_2['tweet_id'].astype(str)

In [ ]:
# Rename columns
df_clean_2 = df_clean_2.rename(columns = {'p1': 'prediction_image_1', 'p2': 'prediction_image_2', 'p3': 'prediction_image_3', 'p1_conf': 'prediction_confidence_1', 'p2_conf': 'prediction_confidence_2', 'p3_conf': 'prediction_confidence_3', 'p1_dog': 'prediction_breed_1', 'p2_dog': 'prediction_breed_2', 'p3_dog': 'prediction_breed_3'})
df_clean_2.head()

In [ ]:
# Make capitalization consistent columns p1, p2, p3 values
prediction_columns = ['prediction_image_1', 'prediction_image_2', 'prediction_image_3']
for p in prediction_columns:
    df_clean_2[p] = df_clean_2[p].str.lower()

In [ ]:
# Replace hyphen with underscore sign in column values
prediction_columns = ['prediction_image_1', 'prediction_image_2', 'prediction_image_3']
for p in prediction_columns:
    df_clean_2[p] = df_clean_2[p].str.replace('-', '_', regex = True)

In [ ]:
# Concatenate/Merge prediction, confidence and breed columns
df_clean_2['prediction_image'] = df_clean_2['prediction_image_1'].map(str) + ' | ' + df_clean_2['prediction_image_2'].map(str) + ' | ' + df_clean_2['prediction_image_3'].map(str)
df_clean_2['prediction_confidence'] = df_clean_2['prediction_confidence_1'].map(str) + ' | ' + df_clean_2['prediction_confidence_2'].map(str) + ' | ' + df_clean_2['prediction_confidence_3'].map(str)
df_clean_2['prediction_breed'] = df_clean_2['prediction_breed_1'].map(str) + ' | ' + df_clean_2['prediction_breed_2'].map(str) + ' | ' + df_clean_2['prediction_breed_3'].map(str)

# Ditch additional columns
ditch_columns = ['prediction_image_1', 'prediction_image_2', 'prediction_image_3', 'prediction_confidence_1', 'prediction_confidence_2', 'prediction_confidence_3', 'prediction_breed_1', 'prediction_breed_2', 'prediction_breed_3']
df_clean_2.drop(ditch_columns, axis = 1, inplace = True)

### Test

In [ ]:
list(df_clean_2)

In [ ]:
df_clean_2.sample(10)

In [ ]:
df_clean_2.head()

#### Create a copy of the tweet_json dataframe

In [ ]:
df_clean_3 = df_3.copy()

### Define
- Convert created_at column into datetime
- Convert id, id_str, in_reply_to_status_id, in_reply_to_status_id_str, in_reply_to_user_id, in_reply_to_user_id_str to string datatype
- Ditch extraneous columns and the ones with most of the values missing
- Filter tweets and get rid of those beyond August 1st, 2017 which do not have image predictions
- Rename id into tweet_id

### Code

In [ ]:
# Convert created_at into datetime type
df_clean_3.created_at = pd.to_datetime(df_clean_3.created_at)

In [ ]:
# Convert id, id_str, in_reply_to_status_id, in_reply_to_status_id_str, in_reply_to_user_id, in_reply_to_user_id_str to string
id_columns = ['id', 'id_str', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str']
for i in id_columns:
    df_clean_3[i] = df_clean_3[i].astype(str)

In [ ]:
# Ditch bias and extraneous columns
drop_columns = ['id_str', 'retweeted', 'display_text_range', 'entities', 'extended_entities', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_screen_name', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'possibly_sensitive', 'possibly_sensitive_appealable', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status', 'source', 'full_text', 'truncated', 'retweeted_status', 'favorited', 'lang']
df_clean_3.drop(drop_columns, axis = 1, inplace = True)

In [ ]:
# Filter tweets and get rid of those beyond August 1st, 2017
df_clean_3.loc[df_clean_3['created_at'] <= '2017-08-01']

In [ ]:
# Rename id column into tweet_id
df_clean_3 = df_clean_3.rename(columns = {'id': 'tweet_id'})

# Reorder columns
df_clean_3 = df_clean_3.iloc[:, [1,3,2,0]]

### Test

In [ ]:
df_clean_3.created_at.dtypes

In [ ]:
df_clean_3.head()

In [ ]:
df_clean_3.columns

In [ ]:
df_clean_3.created_at.sort_values()

In [ ]:
df_clean_3.columns

#### Creating a tidy master dataset from df_clean_retweet, df_clean_2 and df_clean_3

In [ ]:
# Merging the three datasets into one


In [ ]:
df_clean_retweet.head(1)

In [ ]:
df_clean_2.head(1)

In [ ]:
df_clean_3.head(1)

In [ ]:
master_df = df_clean_retweet.merge(df_clean_2, on = 'tweet_id').merge(df_clean_3, on = 'tweet_id')

In [ ]:
master_df.head()

In [ ]:
master_df.shape

In [ ]:
# Check duplicates
master_df.duplicated().sum()

# Storing Data

In [ ]:
master_df.to_csv('twitter_archive_master.csv', index = False)

# Analyze and Visualize Data

## Insights

We will be exploring and analyzing the following insights
- __Which dog rating is most popular ?__
- __Is there a relationship between dog_name and favorite_count ?__
- __What are the most popular dog breeds ?__

In [ ]:
master_df.hist(figsize = (15, 8))

Rating numerator, denominator and retweet_count are the most skewed to the right

### __Which rating is most popular__?

In [ ]:
master_df.rating_numerator.value_counts().plot(kind = 'bar', figsize = (15, 8));

The three most popular and recurring ratings are __12__, __10__ and __11__

### What is the relationship between favorite_count and creation date if any?

In [ ]:
master_df.plot.scatter(x = 'created_at', y = 'favorite_count', figsize = (20, 10))

There seems to be a positive correlation between favorite_count and the date of creation.\
The later the creation, the greater the count

### __What are the most popular dog stages__?

In [ ]:
master_df.dog_stage.value_counts().plot(kind = 'bar', figsize = (15, 5));

The three most popular dog stages are __pupper__, __doggo__, and __puppo__